# Chapter 41: Threading Synchronization

This notebook covers Python's threading synchronization primitives from the `threading` module. You will learn how to coordinate access to shared resources using locks, events, semaphores, barriers, and conditions to write correct concurrent programs.

## Key Concepts
- **`Lock`**: Mutual exclusion -- only one thread can hold the lock at a time
- **`RLock`**: Reentrant lock that the same thread can acquire multiple times
- **`Event`**: Simple signaling mechanism between threads
- **`Semaphore`**: Limits the number of threads accessing a resource concurrently
- **`Barrier`**: Synchronization point where a fixed number of threads must arrive before any proceed
- **`Condition`**: Advanced coordination combining a lock with wait/notify signaling

## Section 1: Lock -- Mutual Exclusion

A `Lock` ensures that only one thread at a time can execute a critical section. Without a lock, concurrent increments to a shared counter can produce incorrect results due to race conditions.

In [ ]:
import threading

# Shared mutable state
lock: threading.Lock = threading.Lock()
counter: list[int] = [0]


def increment() -> None:
    """Increment a shared counter 1000 times with lock protection."""
    for _ in range(1000):
        with lock:
            counter[0] += 1


# Launch 4 threads, each incrementing 1000 times
threads: list[threading.Thread] = [
    threading.Thread(target=increment) for _ in range(4)
]
for t in threads:
    t.start()
for t in threads:
    t.join()

print(f"Final counter value: {counter[0]}")
print(f"Expected: 4000")
print(f"Correct: {counter[0] == 4000}")

In [ ]:
import threading

# Lock can also be acquired and released manually
lock: threading.Lock = threading.Lock()

# acquire() returns True if the lock was successfully acquired
acquired: bool = lock.acquire()
print(f"Lock acquired: {acquired}")

# Non-blocking acquire attempt while lock is held
second_attempt: bool = lock.acquire(blocking=False)
print(f"Second acquire (non-blocking): {second_attempt}")

# Release the lock
lock.release()
print(f"Lock released")

# Now it can be acquired again
third_attempt: bool = lock.acquire(blocking=False)
print(f"Third acquire after release: {third_attempt}")
lock.release()

## Section 2: RLock -- Reentrant Locking

An `RLock` (reentrant lock) can be acquired multiple times by the same thread without deadlocking. This is useful when a function that holds a lock calls another function that also needs the same lock.

In [ ]:
import threading

# RLock allows the same thread to acquire it multiple times
rlock: threading.RLock = threading.RLock()

with rlock:
    print("First level acquired")
    with rlock:  # This would deadlock with a regular Lock!
        print("Second level acquired (reentrant)")
        acquired: bool = True

print(f"Successfully nested: {acquired}")

In [ ]:
import threading

# Practical RLock example: nested method calls
class SafeCounter:
    """A thread-safe counter using RLock for reentrant access."""

    def __init__(self) -> None:
        self._lock: threading.RLock = threading.RLock()
        self._value: int = 0

    def increment(self) -> None:
        """Increment the counter by 1."""
        with self._lock:
            self._value += 1

    def add(self, n: int) -> None:
        """Add n to the counter by calling increment n times."""
        with self._lock:  # Outer lock
            for _ in range(n):
                self.increment()  # Inner lock -- needs RLock!

    def get_value(self) -> int:
        """Return the current counter value."""
        with self._lock:
            return self._value


counter: SafeCounter = SafeCounter()
counter.add(5)
print(f"Counter value after add(5): {counter.get_value()}")

counter.increment()
print(f"Counter value after increment(): {counter.get_value()}")

## Section 3: Event -- Thread Signaling

An `Event` is a simple signaling mechanism. One thread can wait for an event to be set by another thread. This is useful for coordinating startup, shutdown, or notification between threads.

In [ ]:
import threading

event: threading.Event = threading.Event()
result: list[str] = []


def waiter() -> None:
    """Wait for the event to be set, then record completion."""
    event.wait(timeout=5)
    result.append("done")


t: threading.Thread = threading.Thread(target=waiter)
t.start()

# Signal the waiting thread
event.set()
t.join()

print(f"Result: {result}")
print(f"Event is set: {event.is_set()}")

In [ ]:
import threading
import time

# Event can be cleared and reused
event: threading.Event = threading.Event()
log: list[str] = []


def setup_worker() -> None:
    """Simulate setup, then signal readiness."""
    log.append("setup started")
    time.sleep(0.05)  # Simulate setup work
    log.append("setup complete")
    event.set()


def main_worker() -> None:
    """Wait for setup to finish, then proceed."""
    event.wait(timeout=5)
    log.append("main started")


t1: threading.Thread = threading.Thread(target=setup_worker)
t2: threading.Thread = threading.Thread(target=main_worker)
t2.start()
t1.start()
t1.join()
t2.join()

print(f"Execution log: {log}")
print(f"Setup completed before main: {'setup complete' in log and log.index('setup complete') < log.index('main started')}")

## Section 4: Semaphore -- Limiting Concurrent Access

A `Semaphore` limits the number of threads that can concurrently access a resource. It maintains an internal counter that is decremented on each `acquire()` and incremented on each `release()`.

In [ ]:
import threading
import time

# Allow at most 2 concurrent workers
sem: threading.Semaphore = threading.Semaphore(2)
max_concurrent: list[int] = [0]
current: list[int] = [0]
lock: threading.Lock = threading.Lock()


def worker(worker_id: int) -> None:
    """Simulate work while tracking concurrent access."""
    with sem:
        with lock:
            current[0] += 1
            if current[0] > max_concurrent[0]:
                max_concurrent[0] = current[0]
        time.sleep(0.01)  # Simulate work
        with lock:
            current[0] -= 1


threads: list[threading.Thread] = [
    threading.Thread(target=worker, args=(i,)) for i in range(6)
]
for t in threads:
    t.start()
for t in threads:
    t.join()

print(f"Max concurrent workers: {max_concurrent[0]}")
print(f"Semaphore limit respected: {max_concurrent[0] <= 2}")

In [ ]:
import threading
import time

# BoundedSemaphore prevents releasing more than acquired
bsem: threading.BoundedSemaphore = threading.BoundedSemaphore(3)
access_log: list[str] = []
log_lock: threading.Lock = threading.Lock()


def limited_worker(worker_id: int) -> None:
    """Simulate a connection pool with bounded semaphore."""
    with bsem:
        with log_lock:
            access_log.append(f"worker-{worker_id} entered")
        time.sleep(0.01)
        with log_lock:
            access_log.append(f"worker-{worker_id} exited")


threads: list[threading.Thread] = [
    threading.Thread(target=limited_worker, args=(i,)) for i in range(5)
]
for t in threads:
    t.start()
for t in threads:
    t.join()

print(f"Total access events: {len(access_log)}")
print(f"All workers completed: {len([e for e in access_log if 'exited' in e]) == 5}")

## Section 5: Barrier -- Synchronizing Thread Groups

A `Barrier` ensures that a fixed number of threads all reach a synchronization point before any of them proceed. This is useful for phased computations where all workers must finish one phase before starting the next.

In [ ]:
import threading

# All 3 threads must arrive at the barrier before any proceed
barrier: threading.Barrier = threading.Barrier(3)
arrivals: list[int] = []
lock: threading.Lock = threading.Lock()


def worker(worker_id: int) -> None:
    """Wait at barrier, then record arrival."""
    barrier.wait()
    with lock:
        arrivals.append(worker_id)


threads: list[threading.Thread] = [
    threading.Thread(target=worker, args=(i,)) for i in range(3)
]
for t in threads:
    t.start()
for t in threads:
    t.join()

print(f"Arrivals after barrier: {sorted(arrivals)}")
print(f"All 3 threads passed: {len(arrivals) == 3}")

In [ ]:
import threading
import time

# Barrier with phases: threads synchronize between phases
barrier: threading.Barrier = threading.Barrier(3)
phase_log: list[str] = []
lock: threading.Lock = threading.Lock()


def phased_worker(worker_id: int) -> None:
    """Perform two phases, synchronizing between them."""
    # Phase 1
    with lock:
        phase_log.append(f"worker-{worker_id}-phase-1")
    barrier.wait()
    # Phase 2 -- all threads have completed phase 1
    with lock:
        phase_log.append(f"worker-{worker_id}-phase-2")


threads: list[threading.Thread] = [
    threading.Thread(target=phased_worker, args=(i,)) for i in range(3)
]
for t in threads:
    t.start()
for t in threads:
    t.join()

# Count phase entries
phase_1_entries: list[str] = [e for e in phase_log if "phase-1" in e]
phase_2_entries: list[str] = [e for e in phase_log if "phase-2" in e]

print(f"Phase 1 completions: {len(phase_1_entries)}")
print(f"Phase 2 completions: {len(phase_2_entries)}")
print(f"Execution log: {phase_log}")

## Section 6: Condition -- Advanced Coordination

A `Condition` combines a lock with the ability to wait for a notification. Threads can `wait()` until another thread calls `notify()` or `notify_all()`. This is more flexible than `Event` because it is paired with a lock and supports multiple notifications.

In [ ]:
import threading

# Condition variable for producer-consumer coordination
condition: threading.Condition = threading.Condition()
shared_data: list[int] = []
consumed: list[int] = []


def producer() -> None:
    """Produce items and notify the consumer."""
    for i in range(5):
        with condition:
            shared_data.append(i)
            condition.notify()


def consumer() -> None:
    """Consume items, waiting for notifications."""
    for _ in range(5):
        with condition:
            while not shared_data:
                condition.wait()
            item: int = shared_data.pop(0)
            consumed.append(item)


t1: threading.Thread = threading.Thread(target=producer)
t2: threading.Thread = threading.Thread(target=consumer)
t2.start()
t1.start()
t1.join()
t2.join()

print(f"Consumed items: {consumed}")
print(f"All items consumed: {consumed == [0, 1, 2, 3, 4]}")

In [ ]:
import threading

# Condition with wait_for predicate -- cleaner than a while loop
condition: threading.Condition = threading.Condition()
ready: list[bool] = [False]
message: list[str] = [""]


def sender() -> None:
    """Set the message and notify."""
    with condition:
        message[0] = "Hello from sender"
        ready[0] = True
        condition.notify_all()


def receiver(receiver_id: int) -> None:
    """Wait for the message to be ready."""
    with condition:
        condition.wait_for(lambda: ready[0])
        print(f"Receiver {receiver_id} got: {message[0]}")


receivers: list[threading.Thread] = [
    threading.Thread(target=receiver, args=(i,)) for i in range(3)
]
for r in receivers:
    r.start()

sender_thread: threading.Thread = threading.Thread(target=sender)
sender_thread.start()
sender_thread.join()
for r in receivers:
    r.join()

print(f"All receivers notified successfully")

## Section 7: Choosing the Right Primitive

Each synchronization primitive has a specific use case. Choosing the right one makes your code simpler and more correct.

In [ ]:
# Quick reference for when to use each primitive
primitives: dict[str, str] = {
    "Lock": "Protect a shared resource from concurrent access",
    "RLock": "Same as Lock, but allows reentrant (nested) acquisition",
    "Event": "Signal between threads (one-shot or resettable flag)",
    "Semaphore": "Limit N concurrent accesses (e.g., connection pool)",
    "Barrier": "Wait until N threads all reach the same point",
    "Condition": "Wait for a complex condition with lock + notify",
}

for name, use_case in primitives.items():
    print(f"{name:12s} -> {use_case}")

## Summary

### Synchronization Primitives
- **`Lock`**: Use `with lock:` to protect critical sections; only one thread enters at a time
- **`RLock`**: Reentrant lock that the same thread can acquire multiple times without deadlock
- **`Event`**: Call `event.set()` to wake threads waiting on `event.wait()`
- **`Semaphore(n)`**: Allows up to `n` threads to hold the semaphore concurrently
- **`Barrier(n)`**: Blocks each thread at `barrier.wait()` until all `n` threads have arrived
- **`Condition`**: Combines a lock with `wait()` / `notify()` for complex coordination

### Best Practices
- Always use the `with` statement for locks and semaphores to guarantee release
- Prefer `RLock` when methods that hold a lock call other methods needing the same lock
- Use `Event` for simple one-time or resettable signals; use `Condition` for complex predicates
- `BoundedSemaphore` prevents accidental over-releasing
- Keep critical sections as short as possible to minimize contention